In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from scipy import sparse
import anndata
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcdefaults()

/data/public/dulin/miniconda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def cellbingem2adata(gem_path):
    df = pd.read_table(gem_path,comment='#')
    # df.rename(columns={'imagecol':'y'},inplace=True)
    # df.rename(columns={'imagerow':'x'},inplace=True)
    df.rename(columns={'label':'cellID'},inplace=True)

    ex_group = df.groupby(['cellID'])['x','y'].mean().reset_index()
    ex_group.drop(columns= 'cellID',inplace=True)
    ex_group["x"] = ex_group["x"].astype(int)
    ex_group["y"] = ex_group["y"].astype(int)
    df = df[['cellID', 'geneID', 'MIDCount']]
    cell_list = df["cellID"].astype('category')
    gene_list = df["geneID"].astype('category')
    data =  df["MIDCount"].to_numpy()
    row = cell_list.cat.codes.to_numpy()
    col = gene_list.cat.codes.to_numpy()
    obs = pd.DataFrame(index = cell_list.cat.categories)
    var = pd.DataFrame(index = gene_list.cat.categories)
    X = sparse.csr_matrix((data, (row, col)), shape = (len(obs), len(var)))
    adata = anndata.AnnData(X, obs = obs, var = var)
    adata.obsm['spatial'] = np.array(list(ex_group.apply(list,axis = 1)))
    return adata

In [3]:
adata = cellbingem2adata('./input/Visium_to_gem/demoe_visium_orchid.cellbin.gem')
adata

/tmp/ipykernel_25742/3047599902.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ex_group = df.groupby(['cellID'])['x','y'].mean().reset_index()
/data/public/dulin/miniconda/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 2379 × 17486
    obsm: 'spatial'

In [4]:
adata.write("demoe_visium_orchid.cellbin.h5ad")